In [1]:
# CPU USAGE

import multiprocessing

num_cores = multiprocessing.cpu_count()
cores_you_need_to_be_free = 2
cores_to_use = num_cores - cores_you_need_to_be_free
print('cores available:', num_cores,
      '\ncores to be used:', cores_to_use
     )

cores available: 8 
cores to be used: 6


# Download a web page

In [2]:
import requests
import pandas as pd

In [3]:
#Requests allows to send HTTP requests to website/servers. It sends back a response code, and the full html (if successful).

In [4]:
URL = 'https://www.lyrics.com/artist/Linkin-Park/447095'#'https://www.lyrics.com/artist/Imagine-Dragons/1184089'#, 'https://www.lyrics.com/artist/Bob-Marley/2907']#,
      #  'https://www.lyrics.com/artist/Linkin-Park/447095'] 


In [5]:
headers = {'User-agent': 'Mozilla/5.0 (X11; Linux i686; rv:2.0b10) Gecko/20100101 Firefox/4.0b10'}


## Write HTML into text file   

In [19]:
import time
import numpy as np
import os
from bs4 import BeautifulSoup
import re

In [7]:
if os.path.exists('html_data/artist.txt'):
    os.remove('html_data/artist.txt')
#for i in range(len(URL)):
response =requests.get(url=URL, headers=headers)
if response:
    print(response.status_code)
with open('html_data/artist.txt', 'a') as file:
    file.write(response.text)
time.sleep(5)
                


200


## Regular Expression

In [8]:
## open the file contenaining HTML for read 

## Beautiful Soup

In [9]:
#pip install beautifulsoup4

In [10]:
#Convert the raw HTML string to a BeautifulSoup object, so that we can parse the data.

In [11]:
#open text file contenaining  artist html
with open ('html_data/artist.txt') as file:
    artist_soup = BeautifulSoup(file, 'html.parser')


In [12]:
#print(song_soup) ## file is too large to be printed -- return "IOPub data rate exceeded."

In [13]:
#song_soup.body.find('strong').get_text() ## strong contains artists and links to lyrics

### Extract lyrics link

In [14]:
links=[]
for link in artist_soup.body.find_all('strong'):
    if link.a is not None:
        links.append(link.a['href'])

In [16]:
#links

### Download lyrics into file 


In [19]:
if os.path.exists('html_data/songs_html.txt'):
    os.remove('html_data/songs_html.txt')

for i in range(0,len(links)):
    if len(links)<=100:
        response =requests.get(url=URL+links[i], headers=headers)
        if response.status_code != 200:
            print (response.status_code)
        with open('html_data/songs_html.txt', 'a') as file:
            file.write(response.text)
        time.sleep(2)
    else:
        if i < 100:
            response =requests.get(url=URL+links[i], headers=headers)
            if response.status_code != 200:
                print(response.status_code)
            with open('html_data/songs_html.txt', 'a') as file:
                file.write(response.text)
            time.sleep(2)
            

In [ ]:
#song_soup.body.find_all('strong')

### Extract list of artist into a list 

In [12]:
#[artist.text for artist in artist_soup.find_all(class_="artist")]
artist_list = []
for artist in artist_soup.find_all(class_="artist"):
    artist_list.append(artist.text)

In [13]:
artist_list[0]

'Linkin Park'

### Extract lyrics from HTML file

In [23]:
with open ('html_data/songs_html.txt') as file:
    song_soup = BeautifulSoup(file, 'html.parser')

In [32]:
lyrics=[]
for song in song_soup.find_all(id='lyric-body-text'):
    lyrics.append(song.text)

In [37]:
len(lyrics)

100

###  Create pandas dataframe 

In [ ]:
lyric_file = pd.Dataframe({'artist':artist_list[0], 'lyrics': lyrics})

### Define a function

In [25]:
def extract_artist (url):
    
    # function return artist and links to download lyrics
    
    link = url
    headers = {'User-agent': 'Mozilla/5.0 (X11; Linux i686; rv:2.0b10) Gecko/20100101 Firefox/4.0b10'}
    
# extract artist page and store in txt file 

    if os.path.exists('html_data/artist_html.txt'):
        os.remove('html_data/artist_html.txt')
        
    response =requests.get(url=link, headers=headers)
    if response != 200:
        print(response.status_code)
    with open('html_data/artist.txt', 'a') as file:
        file.write(response.text)
    time.sleep(2)
            
# open the file created and sparse data with beautiful soup

    with open ('html_data/artist.txt') as file:
        artist_soup = BeautifulSoup(file, 'html.parser')
        
# Extract link to lyrics

    links=[]
    for lk in artist_soup.body.find_all('strong'):
        if lk.a is not None:
            links.append(lk.a['href'])
            
# Extract artist 
    artist_list = []
    for artist in artist_soup.find_all(class_="artist"):
        artist_list.append(artist.text)
            
# return artist
    return artist_list[0], links
    

In [26]:
def extract_lyrics (url, link):
    # function return the lyrics of the artists
    URL= url
    links = link
    headers = {'User-agent': 'Mozilla/5.0 (X11; Linux i686; rv:2.0b10) Gecko/20100101 Firefox/4.0b10'}
   #check if file exist - if yes  suppress it 

    if os.path.exists('html_data/songs_html.txt'):
        os.remove('html_data/songs_html.txt')
        
     # download lyrics 
    
    for i in range(0,len(links)):
        
        if len(links)<=100:
            response =requests.get(url=URL+links[i], headers=headers)
            if response.status_code != 200:
                print (response.status_code)
            with open('html_data/songs_html.txt', 'a') as file:
                file.write(response.text)
            time.sleep(2)
        else:
            if i < 100:
                response =requests.get(url=URL+links[i], headers=headers)
                if response.status_code != 200:
                    print(response.status_code)
                with open('html_data/songs_html.txt', 'a') as file:
                    file.write(response.text)
                time.sleep(2)
                
                
    # read lyrics with Beautiful soup 
    
    with open ('html_data/songs_html.txt') as file:
        song_soup = BeautifulSoup(file, 'html.parser')
        
    # Extract lyrics in a list 
    lyrics=[]
    for song in song_soup.find_all(id='lyric-body-text'):
        n_jobs=cores_to_use
        lyrics.append(song.text)
        
    return lyrics
    

In [36]:
def create_dataframe(artist, lyrics):
    
    #data = pd.DataFrame(columns=['artist', 'lyrics'])
    
    da = pd.DataFrame({'artist':artist, 'lyrics':lyrics})
    #da.append(data, ignore_index=True)
    return da

In [28]:
url='https://www.lyrics.com/artist/Imagine-Dragons/1184089'

In [29]:
artist , link = extract_artist (url)

200


In [30]:
artist

'Imagine Dragons'

In [31]:
#link

In [32]:
lyric = extract_lyrics (url, link)

In [33]:
#lyric

In [37]:
data = create_dataframe(artist, lyric)

In [38]:
data

,artist,lyrics
0,Imagine Dragons,I'm calling on the God of Jacob\nWhose love en...
1,Imagine Dragons,You leapt from crumbling bridges watching city...
2,Imagine Dragons,"Load up on guns, bring your friends\nIt's fun ..."
3,Imagine Dragons,Love is just a history that they may prove\nAn...
4,Imagine Dragons,Love is just a history that they may prove\nAn...
...,...,...
95,Imagine Dragons,I'm still a fan even though I was salty\nHate ...
96,Imagine Dragons,I am a little unstable\nLoose wires always get...
97,Imagine Dragons,We were speeding together down the dark avenue...
98,Imagine Dragons,We were speeding together down the dark avenue...


In [43]:
artist= song_soup.body.find('strong').get_text()
artist

'Imagine Dragons'

In [41]:
artists = song_soup.find_all('strong')

In [44]:
#find all artist
#artists 

In [69]:
f= song_soup.find('strong', text='Bob Marley')

In [70]:
song_soup.find('strong').find('a')



In [ ]:
#song_soup.body.find_all('strong')[0].text

In [1]:
#print (test)


None


## Example - How to extract information from Webpage

In [13]:
URL = 'https://www.lyrics.com/lyric/34200587/Lana+Del+Rey/'

In [ ]:
headers = {'User-agent': 'Mozilla/5.0 (X11; Linux i686; rv:2.0b10) Gecko/20100101 Firefox/4.0b10'}

In [ ]:
Use a sleep or waiting time between requests, a computer can be much faster than a human

In [ ]:
import time
import numpy as np

In [ ]:
time.sleep(5)
responses = requests.get(url=URL, headers=headers)

In [ ]:
responses

In [ ]:
response.status_code

# Writing/reading files in python

In [ ]:
# w: write creates a file if it doesn't exist or overwrites it if it does:

In [ ]:
f = open('lana_html.txt', 'w')
f.write(responses.text)
f.close()


In [ ]:
# r: read only, you can read in a file and save it as a variable in your code or print it

In [ ]:
f = open('lana_html.txt', 'r')
lana_html_read = f.read()
#print(Bob_html_read)
f.close()

In [ ]:
# a: append mode adds text to the end of a file without overwriting it.

f = open('lana_html.txt', 'a')
f.write("")
f.close()
